In [1]:
import cv2
import mediapipe as mp
import numpy as np
import threading
import tensorflow as tf
gpu=tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0],True)

In [2]:
def make_landmark_timestep(results):
    c_lm = []
    for id, lm in enumerate(results.multi_hand_landmarks[0].landmark):
        if id==0:
            continue
        else:
            c_lm.append(lm.x)
            c_lm.append(lm.y)
            c_lm.append(lm.z)
    return c_lm

def draw_class_on_image(label, frame):
    font = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10, 30)
    fontScale = 1
    fontColor = (0, 255, 0)
    thickness = 2
    lineType = 2
    cv2.putText(frame, str(label),
                bottomLeftCornerOfText,
                font,
                fontScale,
                fontColor,
                thickness,
                lineType)
    return frame

def detect(model, lm_list):
    global label
    lm_list = np.array(lm_list)
    lm_list = np.expand_dims(lm_list, axis=0)
    # print(lm_list.shape)
    results = model.predict(lm_list)
    # print(np.argmax(results))
    char_list = ['A','B','C','D','E',
                'F','G','H','I','J',
                'K','L','M','N','null','O',
                'P','Q','R','S','space','T',
                'U','V','W','X','Y','Z']
    label = np.argmax(results)
    label = char_list[label], label
    return label

In [3]:
label = "Warmup...."
n_time_steps = 15
lm_list = []

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

model = tf.keras.models.load_model("model.h5")

cap = cv2.VideoCapture(0)

i = 0
warmup_frames = 40

while True:

    success, frame = cap.read()
    frame = cv2.flip(frame, 1)

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame)
    i = i + 1
    if i > warmup_frames:
        print("Start detect....")

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

            c_lm = make_landmark_timestep(results)

            lm_list.append(c_lm)
            print(len(lm_list))
            if len(lm_list) == n_time_steps:
                # predict
                t1 = threading.Thread(target=detect, args=(model, lm_list,))
                t1.start()
                lm_list = []

    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    frame = draw_class_on_image(label, frame)
    cv2.imshow("HandSign Recognition", frame)
    cv2.imshow("ASL", cv2.imread('ASL.jpg'))

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Start detect....
1
Start detect....
2
Start detect....
3
Start detect....
4
Start detect....
5
Start detect....
6
Start detect....
7
Start detect....
8
Start detect....
9
Start detect....
10
Start detect....
11
Start detect....
12
Start detect....
13
Start detect....
14
Start detect....
15
Start detect....
1
Start detect....
2
Start detect....
3
Start detect....
4
Start detect....
5
Start detect....
6
Start detect....
7
Start detect....
Start detect....
Start detect....
Start detect....
Start detect....
Start detect....
Start detect....
Start detect....
Start detect....
Start detect....
Start detect....
8
Start detect....
9
Start detect....
Start detect....
10
Start detect....
11
Start detect....
12
Start detect....
13
Start detect....
14
Start detect....
15
Start detect....
1
Start detect....
2
Start detect....
3
Start detect....
4
Start detect....
5
Start detect....
6
Start detect....
7
Start detect....
8
Start detect....
9
Start detect....
10
Start detect....
11
Start detect....
12


In [4]:
cap.release()
cv2.destroyAllWindows()